# Série V
Nonlinear Time Series Analysis - Computer Science Master's Degree Course - Federal University of São Paulo (UNIFESP) - Prof. Elbert E. N. Macau

Author: Rafael Leiniö


## Algorithm Implementations

- [Baker Map](https://github.com/rafaelleinio/ntsa/blob/master/ntsa/algorithms/maps/baker.py)
- [Henon Map](https://github.com/rafaelleinio/ntsa/blob/master/ntsa/algorithms/maps/henon.py)
- [Lyapunov Exponents](https://github.com/rafaelleinio/ntsa/blob/master/ntsa/algorithms/maps/lyapunov_exponents.py)

In [1]:
# fix working dir
import pathlib
import os
path = os.path.join(pathlib.Path().absolute(), '../..')
os.chdir(path)

In [2]:
# Lyapuvo Exponents Solver

"""Based on the implementation found [here](https://github.com/cbnfreitas/lyapunov_exponent_map_and_ode)"""

from numpy import *
from numpy.linalg import *

from ntsa.algorithms.maps.map import Map


class LyapunovExponents:
    def __init__(self, map: Map, tolerance: float = 0.00001, max_iterations: int = 1000):
        self.map = map
        self.tolerance = tolerance
        self.max_iterations = max_iterations

    def calculate_from_initial_conditions(self, initial_conditions: ndarray):
        # setup variables
        n = len(initial_conditions)
        x = initial_conditions
        w = eye(n)
        h = zeros(n)
        l = -1

        # Numerical Lyapunov exponents calculation
        for i in range(0, self.max_iterations):
            x_next, w_next = self.map.f(x), self.map.df(x, w)
            w_next = self._orthogonalize_columns(w_next)

            h_next = h + self._log_of_the_norm_of_the_columns(w_next)
            l_next = h_next / (i + 1)

            if norm(l_next - l) < self.tolerance:
                return sort(l_next)

            h = h_next
            x = x_next
            w = self._normalize_columns(w_next)
            l = l_next

        # Max iter reach without a Solution in desired tolerance
        raise ValueError(
            "Lyapunov Exponents calculation did not converge."
            f" Initial conditions = {initial_conditions}"
            f", Tolerance = {self.tolerance}"
            f", Max Iterations = {self.max_iterations}"
        )

    def calculate_over_set_of_initial_conditions(self, set: ndarray):
        solutions = array(
            [
                self.calculate_from_initial_conditions(initial_conditions)
                for initial_conditions in set
            ]
        )
        return apply_along_axis(lambda v: mean(v), 0, solutions)

    def _orthogonalize_columns(self, a):
        q, r = qr(a)
        return q @ diag(r.diagonal())

    def _normalize_columns(self, a):
        return apply_along_axis(lambda v: v / norm(v), 0, a)

    def _log_of_the_norm_of_the_columns(self, a):
        return apply_along_axis(lambda v: log(norm(v)), 0, a)


## Bakers Map

![](https://i.imgur.com/4JkvPaA.png)

In [3]:
# Reference Code Implementation

from numpy import array

from ntsa.algorithms.maps.map import Map


class Baker(Map):
    def __init__(self):
        pass

    def f(self, xy):
        x, y = xy

        if 0 <= y <= 1 / 2:
            return array([1 / 3 * x, 2 * y])
        elif 1 / 2 < y <= 1:
            return array([1 / 3 * x + 2 / 3, 2 * y - 1])
        else:
            raise ValueError("y not between 0 and 1")

    def df(self, xy, w):
        x, y = xy

        if 0 <= y <= 1 / 2:
            jacobian_matrix = array([[1 / 3, 0], [0, 2]])
            return jacobian_matrix @ w
        elif 1 / 2 < y <= 1:
            jacobian_matrix = array([[1 / 3, 0], [0, 2]])
            return jacobian_matrix @ w
        else:
            raise ValueError("y not between 0 and 1")

In [4]:
# Calculating Lyapunov Exponents:

from ntsa.algorithms.maps import LyapunovExponents
from ntsa.algorithms.maps import Baker

baker_le = LyapunovExponents(map=Baker())
baker_solution = baker_le.calculate_from_initial_conditions(array([0.75, 0.75]))
print(baker_solution)

[-1.09861229  0.69314718]


- As we can see, we got the same result as the image above, with `-1.09861229 ≈ -Ln(3)` and `0.69314718 ≈ -Ln(2)`

## Hénon Map

![](https://i.imgur.com/YqhkKJS.png)

In [5]:
# Reference Code Implementation

from numpy import array

from ntsa.algorithms.maps.map import Map


class Henon(Map):
    def __init__(self, a=1.4, b=0.3):
        self.a, self.b = a, b

    def f(self, xy):
        x, y = xy
        return array([self.a - x ** 2 + self.b * y, x])

    def df(self, xy, w):
        x, y = xy
        jacobian_matrix = array([[-2 * x, self.b], [1, 0]])
        return jacobian_matrix @ w


In [6]:
# Calculating Lyapunov Exponents:

from ntsa.algorithms.maps import LyapunovExponents
from ntsa.algorithms.maps import Henon

henon_le = LyapunovExponents(map=Henon())
henon_solution = henon_le.calculate_from_initial_conditions(array([-0.5, 0.25]))
print(henon_solution)

[-1.60978506  0.40581226]


- The values are close, but not exactly the same
- We can use a set of initial values, calculating a set of solutions and get the average as the final solution

In [7]:
# Calculating the mean over a set of initial conditions

initial_solutions =  array([[-1., -1.], [-1., -0.75], [-1., -0.5], [-1., -0.25], [-1., 0.], [-1., 0.25], [-1., 0.5], [-1., 0.75], [-0.75, -1.], [-0.75, -0.75], [-0.75, -0.5], [-0.75, -0.25], [-0.75, 0.], [-0.75, 0.25], [-0.75, 0.5], [-0.75, 0.75], [-0.5, -1.], [-0.5, -0.75], [-0.5, -0.5], [-0.5, -0.25], [-0.5, 0.], [-0.5, 0.25], [-0.5, 0.5], [-0.5, 0.75], [-0.25, -1.], [-0.25, -0.75], [-0.25, -0.5], [-0.25, -0.25], [-0.25, 0.], [-0.25, 0.25], [-0.25, 0.5], [-0.25, 0.75], [0., -1.], [0., -0.75], [0., -0.5], [0., -0.25], [0., 0.], [0., 0.25], [0., 0.5]])

henon_le = LyapunovExponents(map=Henon())
henon_solution = henon_le.calculate_over_set_of_initial_conditions(initial_solutions)
print(henon_solution)

[-1.61985769  0.41588488]


- As we can see now, much more closer to the correct answer!!